In [1]:
import numpy as np

# 1- Lecture des données

In [41]:
def readFile(name):
    return [i[:-2].split('\t') for i in open("recodata/"+name)]
    

In [42]:
u_data = readFile("u.data")
u1_train = readFile("u1.train")
u1_test = readFile("u1.test")
u5_train = readFile("u5.train")
u5_test = readFile("u5.test")

u_links = readFile("u.links")

In [43]:
print(u_data[0])
print(u1_train[0])
print(u1_test[0])
print(u5_train[0])
print(u5_test[0])
print(u_links[0])

['196', '242', '3', '88125094']
['1', '1', '5', '87496575']
['1', '6', '5', '88743197']
['1', '1', '5', '87496575']
['1', '3', '4', '87854296']
['1', '520', '510', '876', '107', '800', '702', '691', '25']


# 2- Modèle SoRec

In [ ]:
MatrixR
MatrixRc
VectorU
VectorV
VectorT
z=
phi=

In [ ]:
somme = 0
for u in len(MatrixR):
    for i in len(u):
        MatrixRc[u][i]=g(z[u]*w[i])
        somme+=(MatrixR[u][i]-MatrixRc[u][i])**2+lamda*(np.linalg.norm)